# load tons of datasets (~60,000 RNAseq samples)

In [110]:
from taigapy import TaigaClient
tc = TaigaClient()

from depmapomics import tracker as track
from depmapomics import expressions

import dalmatian as dm
from gsheets import Sheets
import pandas as pd

MY_ID = '~/.client_secret.json'
MYSTORAGE_ID = "~/.storage.json"

Sheets.from_files(MY_ID, MYSTORAGE_ID)
#autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## CCLE + TCGA

In [117]:
CCLE_expression

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,ERCC-00157,ERCC-00158,ERCC-00160,ERCC-00162,ERCC-00163,ERCC-00164,ERCC-00165,ERCC-00168,ERCC-00170,ERCC-00171
ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,5.053980,...,0.000000,0.000000,0.150560,2.599318,0.887525,0.000000,0.650765,0.042644,0.344828,7.053872
ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,3.832890,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,3.775051,...,0.286881,0.028569,0.505891,3.744161,1.669027,0.000000,1.454176,0.056584,0.757023,8.763445
ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,4.294253,...,0.201634,0.070389,0.516015,3.941106,1.807355,0.028569,1.560715,0.070389,0.941106,9.021757
ACH-000242,7.465648,0.000000,6.435462,2.414136,3.864929,0.831877,7.198003,5.452530,7.112492,4.710944,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000750,4.173127,0.000000,6.400879,1.807355,3.303050,0.014355,0.137504,4.944858,4.528571,4.383359,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-000285,0.097611,0.000000,6.428276,3.257011,4.980482,0.411426,0.124328,0.704872,4.829342,5.393348,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ACH-001858,5.045268,0.000000,6.991749,1.807355,3.270529,0.028569,3.333424,6.819796,5.395063,3.727920,...,0.226509,0.000000,0.536053,2.965323,1.459432,0.000000,1.608809,0.042644,0.790772,8.595146
ACH-001997,5.805292,0.000000,7.792855,2.482848,3.903038,0.028569,4.124328,6.816600,4.458119,3.664483,...,0.201634,0.000000,0.632268,2.648465,1.531069,0.000000,1.257011,0.000000,0.367371,8.399043


In [116]:
# load from taiga public (figshare link)
# load internal expression,
# latest version can be found at https://depmap.org/portal/download/
# can also be loaded like so pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_expression.csv.gz', index_col=0)
CCLE_expression = tc.get(name='internal-21q3-fe4c',
                         file='CCLE_expression_full')  #40,000x1,500


# load  TCGA expression
# this dataset was generated from  ,using this script: 
# caan be found here: pd.read_csv('gs://ccle_default_params/celligner_ex/TCGA_expression.csv.gz', index_col=0)
TCGA_expression = tc.get(name='celligner-input-9827',
                       file='tumor_expression') # 40,000x13,000

No dataset version provided. Using version 16.
No dataset version provided. Using version 1.


In [ ]:
TCGA_expression.columns[TCGA_expression.columns=="ENSG00000183955"]

In [18]:
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: REFSHEET_URL=https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY

In [23]:
len(CCLE_annotation[(CCLE_annotation.blacklist==0)&(CCLE_annotation.version>1)&(CCLE_annotation.datatype=="rna")]) 
# we have .. replicates in CCLE

72

In [35]:
# loading annotations
CCLE_annotation = track.getTracker() # the function uses pygsheets to load this: REFSHEET_URL=https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY
# Sheets.from_files(MY_ID, MYSTORAGE_ID).get(REFSHEET_URL).sheets[0].to_frame(index_col=0)
# you can also get it from pd.read_csv('gs://ccle_default_params/celligner_ex/CCLE_annotation.csv.gz', index_col=0)


# can be loaded from 
# pd.read_csv('gs://ccle_default_params/celligner_ex/TCGA_annotation.csv.gz', index_col=0)
TCGA_annotation = tc.get(name='celligner-input-9827',
                         file='tumor_annotations') # generated manually 

No dataset version provided. Using version 1.


In [39]:
CCLE_annotation = CCLE_annotation.rename(columns={"stripped_cell_line_name": "sample_id", "primary_disease": "disease_type", "subtype": 'disease_subtype'})[['sample_id', 'participant_id', 'sex', 'age', 'primary_site', 'disease_type', "disease_subtype", 'origin', 'mediatype', 'sequencing_type', ]]

In [40]:
TCGA_annotation = TCGA_annotation.rename(columns={'sampleID': 'sample_id', 'age_at_dx':"age", "gender": "sex", "site_id": "site_id", "lineage": "primary_site", "disease": "disease_type", "subtype": "disease_subtype"})[["sample_type", "sample_id", "age", "sex", "site_id", "primary_site", "disease_type", "disease_subtype"]]

## CCLF

In [43]:
cclf_orga_info = tc.get(name='cclf-organoids-c23d', version=1, file='cclf_orga_info')
cclf_orga_info = cclf_orga_info.rename(columns={'Tumor_type': 'tumor_type', 'Sample_type': 'sample_type', 'RNA_Seq_cancertype': 'disease_type', "CCLF_ID": 'sample_id'})[["tumor_type", "sample_type", "disease_type", "sample_id"]]
cclf_orga_rnaseq = tc.get(name='cclf-organoids-c23d', version=1, file='cclf_orga_rnaseq').T # 40,000x24
cclf_orga_info.index = [i.split("_")[1] for i in cclf_orga_info.sample_id]
cclf_orga_rnaseq.index = [i.split('_')[0][:-1] for i in cclf_orga_rnaseq.index]

In [16]:
#cclf other
cclfrna = dm.WorkspaceManager("nci-mimoun-bi-org/CCLF_RNA_2_0").get_samples() #40,000x160

cclfrna_anno = cclfrna[['source_subtype_rna', 'rna_sample_id', 'tumor_subtype']].replace({'NA': np.nan})

cclfrna_annot = Sheets.from_files(MY_ID, MYSTORAGE_ID).get("https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo").sheets[2].to_frame(index_col=0).rename(columns={"Sequencing on Tissue or Cell model? (MT confirm)": 'tissue_type', "Days to First Agg": 'exp_date', "Contamination % (First Agg)": 'contamination', "collection": 'origin', "Original Material Type": 'history'})[['tissue_type', 'exp_date', 'contamination', 'origin', 'history']]
# get it  from https://docs.google.com/spreadsheets/d/1O9IV_v2vMbebkk_KDWu3LdKBQ16c8lThJKiiWvRxMUo and get 

/home/jeremie/miniconda3/envs/gae/lib/python3.8/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
files, failed, _, _, lowqual, _ = await expressions.postProcess("nci-mimoun-bi-org/CCLF_RNA_2_0", "all_samples", samplesetToLoad = "all_samples", compute_enrichment=False,
trancriptLevelCols = ['rsem_transcripts_expected_count', 'rsem_transcripts_tpm'], geneLevelCols = ["rsem_genes_tpm", "rsem_genes_expected_count"])
cclfrna = files['rsem_genes_tpm']

load QC and generate QC report
Low quality samples
['AA02-Tumor-SM-5LB98', 'AA13-Tumor-SM-GXH5W', 'AA22-Tumor-SM-ALFLY', 'AA45-Tumor-SM-EXXLQ', 'AA59-Tumor-SM-DE63O', 'AB018-Tumor-SM-GXH34', 'AB035-Tumor-SM-HPY1S', 'CCLF_Mix_RNASeq_June2016-AA13-Tumor-SM-6DBPD', 'CCLF_Mix_RNASeq_June2016-AA22-Tumor-SM-ALFLX', 'CCLF_Mix_RNASeq_June2016-AA25-Tumor-SM-B4LHK', 'CCLF_Mix_RNASeq_June2016-AB002-Tumor-SM-5LB1D', 'CCLF_Mix_RNASeq_June2016-AB002-Tumor-SM-5LB1O', 'CCLF_Mix_RNASeq_June2016-AB016-Tumor-SM-9QQCX', 'CCLF_Mix_RNASeq_June2016-BT584-Tumor-SM-5CUQF', 'CCLF_Mix_RNASeq_June2016-BT607-Tumor-SM-5CUQR', 'CCLF_Mix_RNASeq_June2016-JL37-Tumor-SM-7HEVC', 'CCLF_Mix_RNASeq_June2016-PEDS117-Tumor-SM-ACMZA', 'CCLF_PEDS1034-Tumor-SM-HUQMU', 'CCLF_PEDS1037-Tumor-SM-HUQMS', 'CCLF_PEDS1041-Tumor-SM-HUQN4', 'CCLF_PEDS1083-Tumor-SM-HUQNP', 'CCLF_RCRF1009-Tumor-SM-HPY1O', 'CY002-Tumor-SM-B21KX', 'CY003-Tumor-SM-C1AHL', 'DW020-Tumor-SM-E8WFU', 'DW023-Tumor-SM-E8WFV', 'JL50-Tumor-SM-AYSAM', 'JL56-Tumor-SM-CTT

/home/jeremie/ccle_processing/depmapomics/expressions.py:389: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  files, renaming = loadFromRSEMaggregate(refworkspace, todrop=failed, filenames=trancriptLevelCols+geneLevelCols,


renaming files
subsetting gene_id columns
subsetting transcript_id columns
missing: 1907 genes
storing files in 
done


## MET500 and PDXs

In [5]:
# met500 
met500_ann = tc.get(name='met500-fc3c', file='met500_ann')
met500_meta = tc.get(name='met500-fc3c', file='met500_meta')
met500_TPM = tc.get(name='met500-fc3c', file='met500_TPM') #20,979x868 matrix

#Novartis_PDX
Novartis_PDX_ann = tc.get(name='pdx-data-3d29', file='Novartis_PDX_ann')
Novartis_PDX_TPM = tc.get(name='pdx-data-3d29', file='Novartis_PDX_TPM').T # 38,087x445

Novartis_PDX_ann = Novartis_PDX_ann.rename(columns={"sampleID": 'sample_id', 'lineage': 'origin', 'subtype': "disease_subtype", "type": "sample_type"}).set_index('sample_id', drop=True)

#pediatric_PDX
pediatric_PDX_ann = tc.get(name='pdx-data-3d29', file='pediatric_PDX_ann')
pediatric_PDX_TPM = tc.get(name='pdx-data-3d29', file='pediatric_PDX_TPM') #80,000x250

No dataset version provided. Using version 1.
No dataset version provided. Using version 1.
No dataset version provided. Using version 1.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.
No dataset version provided. Using version 2.


In [ ]:
met500_meta["origin"] = met500_ann['lineage'].values
met500_meta["primary_site"] = met500_ann['primary_site'].values
del met500_ann

met500_meta = met500_meta.rename(columns={"sampleID": 'sample_id', 'lineage': 'origin', 'subtype': "disease_subtype", "type": "sample_type", "Sex": "sex", 'Phase':'stage', 'sample_source': 'patient_id', 'tc': 'tumor_purity'}).set_index('sample_id', drop=True)[["origin", "patient_id", "tumor_purity", "primary_site"]]

In [70]:
[(i.split('me patient as ')[-1].split(' (')[0],v) if type(i) is str and 'ame patient' in i else '' for v, i in pediatric_PDX_ann[["sampleID","Other_info1"]].values]

[('ALL-105', 'ALL-102'),
 ('ALL-102', 'ALL-105'),
 '',
 '',
 '',
 '',
 ('ALL-102', 'ALL-115'),
 '',
 '',
 '',
 ('ALL-46', 'ALL-121'),
 '',
 ('ALL-58', 'ALL-123'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-121', 'ALL-46'),
 '',
 '',
 '',
 '',
 '',
 ('ALL-123', 'ALL-58'),
 '',
 ('ALL-25', 'ALL-61'),
 '',
 '',
 ('ALL-81', 'ALL-80'),
 ('ALL-80', 'ALL-81'),
 '',
 ('ALL-32', 'ALL-90'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-61', 'ALL-25'),
 '',
 '',
 '',
 ('ALL-90', 'ALL-32'),
 '',
 '',
 '',
 '',
 '',
 '',
 ('ALL-83', 'ALL-82'),
 ('ALL-82', 'ALL-83'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('COG-N-453x', 'COG-N-452x'),
 ('COG-N-452x', 'COG-N-453x'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('COG-N-623x', 'COG-N-603x'),
 ('COG-N-618x', 'COG-N-619x'),
 '',
 ('COG-N-603x', 'COG-N-623x'),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('98

In [ ]:
pediatric_PDX_ann['participant_id'] = pediatric_PDX_ann.index

#created frrom manual inspection
samepatient = [('NCH-CA-2', 'NCH-CA-1'), ('ALL-105', 'ALL-102', "ALL-115"), ('ALL-46', 'ALL-121'), ('ALL-25', 'ALL-61'), ('ALL-81', 'ALL-80'), ('ALL-32', 'ALL-90'), ('ALL-58', 'ALL-123'), ('ALL-82', 'ALL-83'), ("COG-N-623x", "COG-N-603x"), ("COG-N-453x","COG-N-452x"), ("COG-N-618x", "COG-N-619x"), ('22909PNET', '9850PNET'), ('OS-34', 'OS-34-SJ'), ('OS-36', 'OS-36-SJ', 'OS-32'),  ('Rh-30R', 'Rh-30')]

for val in samepatient:
    for i in val[1:]:
        pediatric_PDX_ann.loc[i, 'participant_id']=val[0]
        
pediatric_PDX_ann['age'] = ['adult' if i =='Adult' else 'child' for i in pediatric_PDX_ann['Other_info1']]

pediatric_PDX_ann.rename(columns={"sampleID": 'sample_id', 'lineage': 'origin', 'subtype': "disease_subtype", "type": "sample_type", "Sex": "sex", 'Phase':'stage', 'Site.of.Specimen':'primary_site'}).set_index('sample_id', drop=True)[["sample_id", "origin", "disease_subtype", "sample_type", "sex", "stage", "primary_site"]]

## DUOS datasets

In [ ]:
#https://duos.broadinstitute.org/dataset_catalog

## tumor inf elife

In [ ]:
elife_tumorinf = tc.get(name='tumor-infiltration-3307', version=1, file='elife_tumorinf')

## tirosh's melanoma

In [100]:
melanoma = tc.get(name='tirosh-melanoma-scrnaseq-60f0', file='melanoma')

[##################]100% |  27.0 MiB/s |  72.7 MiB /  72.7 MiB | Time:  0:00:02


In [106]:
melanoma

,Cy72_CD45_H02_S758_comb,CY58_1_CD45_B02_S974_comb,Cy71_CD45_D08_S524_comb,Cy81_FNA_CD45_B01_S301_comb,Cy80_II_CD45_B07_S883_comb,Cy81_Bulk_CD45_B10_S118_comb,Cy72_CD45_D09_S717_comb,Cy74_CD45_A03_S387_comb,Cy71_CD45_B05_S497_comb,Cy80_II_CD45_C09_S897_comb,...,CY75_1_CD45_CD8_7__S265_comb,CY75_1_CD45_CD8_3__S127_comb,CY75_1_CD45_CD8_1__S61_comb,CY75_1_CD45_CD8_1__S12_comb,CY75_1_CD45_CD8_1__S25_comb,CY75_1_CD45_CD8_7__S223_comb,CY75_1_CD45_CD8_1__S65_comb,CY75_1_CD45_CD8_1__S93_comb,CY75_1_CD45_CD8_1__S76_comb,CY75_1_CD45_CD8_7__S274_comb
Cell,,,,,,,,,,,,,,,,,,,,,
tumor,72.0000,58.0000,71.0000,81.0000,80.0000,81.0000,72.0000,74.0000,71.0000,80.00000,...,75.0000,75.0000,75.00000,75.00000,75.0000,75.0000,75.0000,75.0000,75.0000,75.0000
"malignant(1=no,2=yes,0=unresolved)",1.0000,1.0000,2.0000,2.0000,2.0000,2.0000,1.0000,1.0000,2.0000,2.00000,...,1.0000,1.0000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
"non-malignant cell type (1=T,2=B,3=Macro.4=Endo.,5=CAF;6=NK)",2.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,1.0000,0.0000,0.00000,...,1.0000,1.0000,1.00000,1.00000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
C9orf152,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00000,...,0.0000,0.0000,0.00000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
RPS11,9.2172,8.3745,9.3130,7.8876,8.3291,7.8336,8.3737,8.1338,8.4373,7.59680,...,0.0000,7.8639,5.85050,0.62639,6.2734,5.4889,4.9262,7.0958,3.9970,3.9897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PIK3IP1,7.6069,0.0000,0.0000,0.0000,0.0000,0.0000,6.5457,0.0000,0.0000,0.00000,...,0.0000,0.0000,0.00000,0.00000,6.4087,0.0000,3.7384,0.0000,0.0000,6.7535
SNRPD2,0.0000,0.0000,3.9871,5.2639,6.0824,5.6424,0.0000,0.0000,5.9451,6.52150,...,4.4471,4.7607,0.00000,4.74330,5.4041,6.0986,0.0000,0.0000,0.0000,5.9313
SLC39A6,0.0000,0.0000,3.8777,3.7660,1.7816,4.3679,0.0000,0.0000,3.4498,0.43189,...,0.0000,0.0000,0.47546,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,5.2398


In [113]:
melanoma_ann = pd.DataFrame()

typ={1:"normal", 2:"tumor",0: np.nan}
orig={1:"melanoma", 2:"B-cell", 3: "macrophage", 4: "endothelial", 5: "CAF", 6:"NK-cell", 0: np.nan}

melanoma_ann['tumor_prob'] = [int(i) for i in melanoma.loc['tumor']]
melanoma_ann["cell_type"] = [typ[int(i)] for i in melanoma.loc['malignant(1=no,2=yes,0=unresolved)']]
melanoma_ann['origin'] = [orig[int(i)] for i in melanoma.loc['non-malignant cell type (1=T,2=B,3=Macro.4=Endo.,5=CAF;6=NK)']]
melanoma_ann['name'] = [i.split('_')[0] for i in melanoma.columns]
melanoma_ann['sample_ID'] = [i.split('_')[-2] for i in melanoma.columns]

IndexError: list index out of range

In [114]:
for i in melanoma.columns:
    try:
        i.split('_')[-2]
    except:
        print(i)

cy80-Cd45-pos-PD1-pos-B09-S45-comb
cy80-Cd45-pos-Pd1-neg-S366-H06-S366-comb
cy80-Cd45-pos-Pd1-neg-S202-A10-S202-comb
cy80-Cd45-pos-Pd1-neg-S201-A09-S201-comb
cy80-Cd45-pos-Pd1-neg-S221-B05-S221-comb
cy80-Cd45-pos-PD1-pos-A03-S15-comb
cy80-Cd45-pos-PD1-pos-B11-S47-comb
cy80-Cd45-pos-PD1-pos-S189-H09-S189-comb
cy80-Cd45-pos-PD1-pos-A05-S17-comb
cy80-Cd45-pos-PD1-pos-C02-S62-comb
cy80-Cd45-pos-PD1-pos-D12-S96-comb
cy80-Cd45-pos-PD1-pos-S165-G09-S165-comb
cy80-Cd45-pos-PD1-pos-S115-E07-S115-comb
cy80-Cd45-pos-Pd1-neg-S294-E06-S294-comb
cy80-Cd45-pos-PD1-pos-C07-S67-comb
cy80-Cd45-pos-Pd1-neg-S228-B12-S228-comb
cy80-Cd45-pos-Pd1-neg-S370-H10-S370-comb
cy80-Cd45-pos-Pd1-neg-S196-A04-S196-comb
cy80-Cd45-pos-PD1-pos-S120-E12-S120-comb
cy80-Cd45-pos-Pd1-neg-S272-D08-S272-comb
cy80-Cd45-pos-PD1-pos-S159-G03-S159-comb
cy80-Cd45-pos-PD1-pos-S138-F06-S138-comb
cy80-Cd45-pos-PD1-pos-S191-H11-S191-comb
cy80-Cd45-pos-PD1-pos-S192-H12-S192-comb
cy80-Cd45-pos-Pd1-neg-S251-C11-S251-comb
cy80-Cd45-pos-PD1

cy79-p3-CD45-neg-PDL1-neg-G06-S270-comb
cy79-p3-CD45-neg-PDL1-neg-H06-S282-comb
cy79-p3-CD45-neg-PDL1-neg-F03-S255-comb
cy79-p3-CD45-pos-PD1-neg-B11-S119-comb
cy79-p3-CD45-pos-PD1-neg-A09-S105-comb
cy79-p3-CD45-neg-PDL1-neg-G12-S276-comb
cy79-p3-CD45-pos-PD1-neg-G02-S170-comb
cy79-p3-CD45-neg-PDL1-neg-C03-S219-comb
cy79-p3-CD45-neg-PDL1-neg-H05-S281-comb
cy79-p3-CD45-neg-PDL1-neg-E11-S251-comb
cy79-p3-CD45-neg-PDL1-neg-H01-S277-comb
cy79-p3-CD45-pos-PD1-neg-G03-S171-comb
cy79-p3-CD45-pos-PD1-neg-G09-S177-comb
cy79-p3-CD45-pos-PD1-neg-G06-S174-comb
cy79-p3-CD45-neg-PDL1-neg-C01-S217-comb
cy79-p3-CD45-pos-PD1-neg-F12-S168-comb
cy79-p3-CD45-pos-PD1-neg-E07-S151-comb
cy79-p3-CD45-pos-PD1-neg-C10-S130-comb
cy79-p3-CD45-pos-PD1-neg-F08-S164-comb
cy79-p3-CD45-pos-PD1-neg-E02-S146-comb
cy79-p3-CD45-neg-PDL1-neg-H07-S283-comb
cy79-p3-CD45-neg-PDL1-neg-D08-S236-comb
cy79-p3-CD45-neg-PDL1-neg-H11-S287-comb
cy79-p3-CD45-neg-PDL1-neg-G08-S272-comb
cy79-p3-CD45-neg-PDL1-neg-F02-S254-comb
cy79-p3-CD4

## st jude

In [ ]:
pd.read_csv('') # 40,000 x 3500


## encode

In [ ]:
 # 40,000 x 1100

## ICGC

In [ ]:
 # 40,000 x 12,500

## GTEX

In [3]:
from anndata import AnnData, read_h5ad

In [9]:
#! curl https://storage.googleapis.com/gtex_analysis_v9/snrna_seq_data/GTEx_8_tissues_snRNAseq_atlas_071421.public_obs.h5ad --output temp/gtex_8_atlas_public.h5ad

## GTEX additional
https://storage.googleapis.com/gtex_external_datasets/eyegex_data/rna_seq_data/EyeGEx_retina_combined_genelevel_expectedcounts_byrid_nooutlier.tpm.matrix.gct
    
https://storage.googleapis.com/gtex_external_datasets/eyegex_data/annotations/EyeGEx_meta_combined_inferior_retina_summary_deidentified_geo_ids.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1810M  100 1810M    0     0   126M      0  0:00:14  0:00:14 --:--:--  126M


In [10]:
gtex_v9 = read_h5ad("temp/gtex_8_atlas_public.h5ad") #209,126 × 17,695

In [ ]:
gtex_add = # 80,000 x 500

## THEIS LAB scRNAseq datasets

In [ ]:
https://theislab.github.io/sfaira-portal/Datasets #50,000x13,000


## HCMI

In [6]:
# HCMI dataset
# Code to generate this dataset can be found here:
# https://github.com/broadinstitute/hcmi-processing/blob/main/hcmi-rna-analysis-210226.ipynb
hcmi_ltpm = tc.get(name='hcmi-data-ac4b', file='hcmi_ltpm').T # 60486 x 157
hcmi_sample_info = tc.get(name='hcmi-data-ac4b', file='hcmi_sample_info')
#sample_info = tc.get(name='hcmi-data-ac4b', file='sample-info')

No dataset version provided. Using version 7.
No dataset version provided. Using version 7.
No dataset version provided. Using version 7.


## GDSC

In [ ]:
for i in range(1):
    val = "E-MTAB-3610.raw."+i+".zip"
    ! curl https://www.ebi.ac.uk/arrayexpress/files/E-MTAB-3610/$val -o temp/$val
    ! gunzip temp/$val # 40,000x1200

## L1000 dataset

In [97]:
import cmapPy.pandasGEXpress.parse_gct as pg

In [104]:
pg.parse('temp/'+val)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [99]:
folder = "gs://ccle_default_params/celligner_ex/"
for val in ["level5_beta_ctl_n58022x12328.gctx",
           ]:#"level5_beta_trt_cp_n720216x12328.gctx",
            #"level5_beta_trt_misc_n8283x12328.gctx",
            #"level5_beta_trt_oe_n34171x12328.gctx",
            #"level5_beta_trt_sh_n238351x12328.gctx",
            #"level5_beta_trt_xpr_n142901x12328.gctx",]:
    cmd = "gsutil cp " + folder + val + " temp/"
    ! $cmd
    res = pg('temp/'+val)
    

Copying gs://ccle_default_params/celligner_ex/level5_beta_ctl_n58022x12328.gctx...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

- [1 files][  2.7 GiB/  2.7 GiB]   10.5 MiB/s                                   
Operation completed over 1 objects/2.7 GiB.                                      


TypeError: 'module' object is not callable

# QC

## get the same set of overlapping genes

## compute correlation

- find unknown duplicates
- find problematic duplicates

## analyse  annotation files

- find mismatch annotations
- add missing annotations, need: 
    - sequencer, 
    - expression_type, 
    - media, 
    - origin, 
    - tissue type,
    - disease,
    - sub_disease,
    - cell type, 
    - group, 
    - sex, 
    - age, 
    - contamination
    - organism
- find good set of names for annotations

In [20]:
cclfrna_annot['tumor_subtype']

,source_subtype_rna,rna_sample_id,tumor_subtype
sample_id,,,
AA02-Tumor-SM-5LB98,"Cells:Cell Line, Viable",AA02-Tumor-SM-5LB98,Metastatic
AA13-Tumor-SM-GXH5W,Cells:Pellet frozen,AA13-Tumor-SM-GXH5W,Primary
AA22-Tumor-SM-ALFLY,Cells:Growing,CCLF_Mix_RNASeq_June2016-AA22-Tumor-SM-ALFLX,Primary
AA22-Tumor-SM-GXH2L,Cells:Pellet frozen,AA22-Tumor-SM-GXH2L,Primary
AA26-Tumor-SM-AU5YZ,Cells:Pellet frozen,AA26-Tumor-SM-AU5YZ,Primary
...,...,...,...
RP-571_DW036T_v2_RNA_OnPrem,NaN,NaN,NaN
SM045-Tumor-SM-AREHF,Tissue:Fresh Tissue,SM045-Tumor-SM-AREHF,Primary
SP022-Tumor-SM-B289M,"Cells:Cell Line, Viable",SP022-Tumor-SM-B289M,Primary


In [82]:
Novartis_PDX_TPM = Novartis_PDX_TPM.T